# Regression Week 2: Multiple Regression (gradient descent)

In the first notebook we explored multiple regression using Turi Create. Now we will use Turi Create along with numpy to solve for the regression weights with gradient descent.

In this notebook we will cover estimating multiple regression weights via gradient descent. You will:
* Add a constant column of 1's to a Turi Create SFrame to account for the intercept
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

# Fire up Turi Create

Make sure you have the latest version of Turi Create

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = pd.read_csv('kc_house_data.csv')
sales_train = pd.read_csv('kc_house_train_data.csv')
sales_test = pd.read_csv('kc_house_test_data.csv')
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the other Week 2 notebook. For this notebook, however, we will work with the existing features.

# Convert to Numpy Array

Although SFrames offer a number of benefits to users (especially when using Big Data and built-in Turi Create functions) in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 

First we need to take the SFrame of our data and convert it into a 2D numpy array (also called a matrix). To do this we use Turi Create's built in .to_dataframe() which converts the SFrame into a Pandas (another python library) dataframe. We can then use Panda's .as_matrix() to convert the dataframe into a numpy matrix.

Now we will write a function that will accept an SFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
* A numpy array containing the values of the output

With this in mind, complete the following function (where there's an empty line you should write a line of code that does what the comment above indicates)

In [3]:
def get_numpy_data(df, features):
    data = df[features].values
    m = len(data)
    x0 = np.ones((m,))
    data = np.column_stack([x0, data])
    return data

For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [4]:
features_arr = get_numpy_data(sales, 'sqft_living')

# Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 1.0\*1.0 + 1.0\*1180.0 = 1181.0 this is the dot product between these two arrays. If they're numpy arrayws we can use np.dot() to compute this:

In [5]:
my_weights = np.array([1., 1.]) # the example weights
my_features = np.array([1., 1180.0]) # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
predicted_value

1181.0

np.dot() also works when dealing with a matrix and a vector. Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features *matrix* and the weights *vector*. With this in mind finish the following predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [6]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    return feature_matrix @ weights

If you want to test your code run the following cell:

In [7]:
test_predictions = predict_output(features_arr, my_weights)
print (test_predictions[0]) # should be 1181.0
print (test_predictions[1]) # should be 2571.0

1181.0
2571.0


# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)\* [feature_i]

The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [8]:
def RSS(Y, Y_hat):
    err = Y - Y_hat
    rss = err @ err
    return rss

In [9]:
def feature_derivative(X, Y, Y_hat):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    err = Y - Y_hat
    dw = -2 * err
    dw = dw @ X
    return dw

To test your feature derivartive run the following:

In [12]:
example1 = get_numpy_data(sales, 'sqft_living') 
my_weights = np.array([0., 0.])
test_predictions = predict_output(example1, my_weights) 
derivative = feature_derivative(example1, sales['price'].values, test_predictions)
derivative

array([-2.33458500e+10, -5.87888151e+13])

# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [13]:
from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [27]:
def regression_gradient_descent(X, Y, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights)
    while not converged:
        Y_hat = predict_output(X, weights)
        dW = feature_derivative(X, Y, Y_hat)
        weights = weights - step_size * dW
        if np.linalg.norm(dW) < tolerance:
            converged = True
    return weights

A few things to note before we run the gradient descent. Since the gradient is a sum over all the data points and involves a product of an error and a feature the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features. 

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

# Running the Gradient Descent as Simple Regression

First let's split the data into training and test data.

In [24]:
train_y = sales_train['price']
train_X = sales_train.drop('price', axis = 1) 
test_y = sales_test['price']
test_X = sales_test.drop('price', axis = 1) 

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimate the parameters in the simple regression on squarefeet. The folowing cell sets up the feature_matrix, output, initial weights and step size for the first model:

In [25]:
# let's test out the gradient descent
simple_features = 'sqft_living'
simple_features = get_numpy_data(train_X, simple_features)
initial_weights = [-47000., 1.]
step_size = 7e-12
tolerance = 2.5e7

Next run your gradient descent with the above parameters.

In [28]:
W1 = regression_gradient_descent(simple_features, train_y, initial_weights, step_size, tolerance)
W1

array([-46999.88716555,    281.91211918])

How do your weights compare to those achieved in week 1 (don't expect them to be exactly the same)? 

**Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?**

Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [30]:
test_simple_feature_matrix = get_numpy_data(test_X, 'sqft_living')

Now compute your predictions using test_simple_feature_matrix and your weights from above.

In [33]:
np.round(predict_output(test_simple_feature_matrix, W1), 1)

array([356134.4, 784640.9, 435069.8, ..., 663418.7, 604217.1, 240550.5])

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 1 (round to nearest dollar)?**

In [34]:
np.round(predict_output(test_simple_feature_matrix, W1), 2)[0]

356134.44

Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [35]:
RSS(test_y, predict_output(test_simple_feature_matrix, W1))

275400044902128.3

# Running a multiple regression

Now we will use more than one actual feature. Use the following code to produce the weights for a second model with the following parameters:

In [40]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
feature_matrix = get_numpy_data(train_X, model_features)
initial_weights = [-100000., 1., 1.]
step_size = 4e-12
tolerance = 1e9

Use the above parameters to estimate the model weights. Record these values for your quiz.

In [41]:
W2 = regression_gradient_descent(feature_matrix, train_y, initial_weights, step_size, tolerance)
W2

array([-9.99999688e+04,  2.45072603e+02,  6.52795267e+01])

Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [42]:
predict_output(get_numpy_data(test_X, model_features), W2)

array([366651.41162949, 762662.39850726, 386312.09557541, ...,
       682087.39916306, 585579.27901327, 216559.20391786])

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?**

In [43]:
np.round(predict_output(get_numpy_data(test_X, model_features), W2), 2)[0]

366651.41

What is the actual price for the 1st house in the test data set?

In [44]:
round(test_y[0], 2)

310000.0

**Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?**

Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [46]:
RSS(test_y, predict_output(get_numpy_data(test_X, model_features), W2))

270263443629803.5

**Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data? **